In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#loading and observing the data
file_path = '20231204_UpdatedData\INC10447_ED_VISITS_20231201.xlsx'

df_visits = pd.read_excel(file_path)

df_visits.head()

In [ ]:
# checking the columns and their data types

# missing values in this case indicate that the patient did not go through the given stage
df_visits.info()

In [ ]:
df_visits.describe()

In [ ]:
# mean age of patient's who died = 74.284919	
df_with_dod = df_visits[df_visits['DOD'].notnull()]
df_with_dod.describe()

In [ ]:
df_visits.head()

In [ ]:

gender_count = df_visits['SEX'].value_counts()

gender_count

In [ ]:


# Count the number of rows for each clinic
clinic_row_counts = df_visits['FACILITY_NAME'].value_counts()

clinic_row_counts

In [ ]:

# Count the occurrences of each CTAS level
ctas_counts = df_visits['CTAS'].value_counts()

ctas_counts 

In [ ]:
# Count the occurrences of each depart dispositio
disposition_counts = df_visits['DEPART_DISPOSITION_DESC'].value_counts()

disposition_counts

In [ ]:
# List of column pairs to combine
column_pairs = [
    ('ED_SERVICE_DATE', 'ED_SERVICE_TIME'),
    ('ED_TRIAGE_DATE', 'ED_TRIAGE_TIME'),
    ('ED_ADMIT_DATE', 'ED_ADMIT_TIME'),
    ('ED_DISCHARGE_DATE', 'ED_DISCHARGE_TIME'),
    ('ED_ASSESSMENT_DATE', 'ED_ASSESSMENT_TIME'),
    ('ED_ADMIT_DECISION_DATE', 'ED_ADMIT_DECISION_TIME'),
    ('ED_DEPART_DATE', 'ED_DEPART_TIME')
]

for date_col, time_col in column_pairs:
    # Convert date column to string format
    df_visits[date_col + '_str'] = df_visits[date_col].dt.strftime('%Y-%m-%d')
    
    # Attempt to convert time column to datetime format, then to string. 'Coerce' turns errors into NaT.
    df_visits[time_col + '_str'] = pd.to_datetime(df_visits[time_col], errors='coerce', format='%H:%M:%S').dt.time.astype(str)
    
    # Handling cases where time is NaT (resulting in 'NaT' string), replace it with a default '00:00:00'
    df_visits[time_col + '_str'] = df_visits[time_col + '_str'].replace('NaT', '00:00:00')
    
    # Combine date and time strings into the desired datetime format
    combined_str = df_visits[date_col + '_str'] + 'T' + df_visits[time_col + '_str'] + '.000+00:00'
    
    # Convert the combined string into datetime, using 'coerce' to handle any conversion errors (e.g., invalid dates/times)
    df_visits[date_col + '_DATETIME'] = pd.to_datetime(combined_str, errors='coerce', utc=True)
    


In [ ]:
df_visits.head()

In [ ]:
df_visits.info()

In [ ]:
# Define the list of specific datetime columns to format
datetime_columns = [
    'ED_SERVICE_DATE_DATETIME',
    'ED_TRIAGE_DATE_DATETIME',
    'ED_ADMIT_DATE_DATETIME',
    'ED_DISCHARGE_DATE_DATETIME',
    'ED_ASSESSMENT_DATE_DATETIME',
    'ED_ADMIT_DECISION_DATE_DATETIME',
    'ED_DEPART_DATE_DATETIME'
]

# Iterate over the specified datetime columns
for col in datetime_columns:
    # Check if the column is not entirely null
    if df_visits[col].notnull().any():
        # Format the column and create a new one with the formatted datetime string
        formatted_col_name = col + '_str'  # Naming the new column
        df_visits[formatted_col_name] = df_visits[col].dt.strftime('%Y-%m-%dT%H:%M:%S.000') + '+00:00'


In [ ]:
df_visits.head()

In [ ]:
df_visits.info()

In [ ]:
columns_to_drop = [ 'ED_SERVICE_DATE', 'ED_SERVICE_TIME',
    'ED_TRIAGE_DATE', 'ED_TRIAGE_TIME',
    'ED_ADMIT_DATE', 'ED_ADMIT_TIME',
    'ED_DISCHARGE_DATE', 'ED_DISCHARGE_TIME',
    'ED_ASSESSMENT_DATE', 'ED_ASSESSMENT_TIME',
    'ED_ADMIT_DECISION_DATE', 'ED_ADMIT_DECISION_TIME',
    'ED_DEPART_DATE', 'ED_DEPART_TIME',
    'ED_SERVICE_DATE_str', 'ED_SERVICE_TIME_str',
    'ED_TRIAGE_DATE_str', 'ED_TRIAGE_TIME_str',
    'ED_ADMIT_DATE_str', 'ED_ADMIT_TIME_str',
    'ED_DISCHARGE_DATE_str', 'ED_DISCHARGE_TIME_str',
    'ED_ASSESSMENT_DATE_str', 'ED_ASSESSMENT_TIME_str',
    'ED_ADMIT_DECISION_DATE_str', 'ED_ADMIT_DECISION_TIME_str',
    'ED_DEPART_DATE_str', 'ED_DEPART_TIME_str', 'ED_SERVICE_DATE_DATETIME',
    'ED_TRIAGE_DATE_DATETIME',
    'ED_ADMIT_DATE_DATETIME',
    'ED_DISCHARGE_DATE_DATETIME',
    'ED_ASSESSMENT_DATE_DATETIME',
    'ED_ADMIT_DECISION_DATE_DATETIME',
    'ED_DEPART_DATE_DATETIME'
]

# Drop the columns
df_visits.drop(columns=columns_to_drop, inplace=True)
df_visits.info()

In [ ]:
# Convert the formatted datetime string columns back to datetime for comparison
df_visits['ED_SERVICE_DATE_DATETIME'] = pd.to_datetime(df_visits['ED_SERVICE_DATE_DATETIME_str'], errors='coerce', utc=True)
df_visits['ED_TRIAGE_DATE_DATETIME'] = pd.to_datetime(df_visits['ED_TRIAGE_DATE_DATETIME_str'], errors='coerce', utc=True)

# Check cases where service date-time is before triage date-time
cases_service_before_triage = df_visits[df_visits['ED_SERVICE_DATE_DATETIME'] < df_visits['ED_TRIAGE_DATE_DATETIME']]

# Count the number of such cases
num_cases_service_before_triage = len(cases_service_before_triage)

print(f"Number of cases where service date-time is before triage date-time: {num_cases_service_before_triage}")


In [ ]:
# Assuming ED_SERVICE_DATE_DATETIME and ED_TRIAGE_DATE_DATETIME are already in datetime format
# Filter the DataFrame to keep only rows where service datetime is not before triage datetime
df_visits_filtered = df_visits[df_visits['ED_SERVICE_DATE_DATETIME'] >= df_visits['ED_TRIAGE_DATE_DATETIME']]

# df_visits_filtered now contains only the rows where service datetime is the same or after triage datetime
df_visits_filtered.info()

df_visits_filtered['FACILITY_NAME'] = df_visits_filtered['FACILITY_NAME'].str.replace(',', ' -')

In [ ]:
# Drop the '_str' columns if they exist
columns_to_drop = ['ED_SERVICE_DATE_DATETIME', 'ED_TRIAGE_DATE_DATETIME']
df_visits_filtered = df_visits_filtered.drop(columns=columns_to_drop, errors='ignore')

df_visits_filtered.head()


In [ ]:
# melt the table to correct format

df_melted = df_visits_filtered
df_visits_filtered.info()

In [ ]:

df_melted = pd.melt(df_melted, id_vars=['SID', 'SEX', 'DOB', 'DOD', 'ED_POSTAL_CODE', 'FACILITY_ID', 'FACILITY_NAME', 'VISIT_ID', 'VISIT_AGE', 'CTAS', 'REASON_FOR_VISIT', 'PRESENTING_COMPLAINT', 'DEPART_DISPOSITION_ID', 'DEPART_DISPOSITION_DESC'],  
                 value_vars=['ED_SERVICE_DATE_DATETIME_str', 'ED_TRIAGE_DATE_DATETIME_str', 'ED_ADMIT_DATE_DATETIME_str', 'ED_DISCHARGE_DATE_DATETIME_str', 'ED_ASSESSMENT_DATE_DATETIME_str', 'ED_ADMIT_DECISION_DATE_DATETIME_str', 'ED_DEPART_DATE_DATETIME_str'], 
                 var_name='Activity', value_name='Start_Time',
                 ignore_index=True)
df_melted.sample(10)

In [ ]:
# Step 1: Drop rows where Start_Time is NaN
df_melt_clean = df_melted.dropna(subset=['Start_Time'])

# Step 2: Sort by SID first, then by Start_Time
df_melt_sorted = df_melt_clean.sort_values(by=['SID', 'Start_Time'])

# Step 3: (Optional) Reset the index, dropping the old index
df_melt_sorted = df_melt_sorted.reset_index(drop=True)

df_melt_sorted.head()

In [ ]:
df_melt_sorted['Activity'].unique()

In [ ]:
# Define the mapping of old values to new values
value_mapping = {
    'ED_TRIAGE_DATE_DATETIME_str': 'Triage',
    'ED_ASSESSMENT_DATE_DATETIME_str': 'Assessment',
    'ED_SERVICE_DATE_DATETIME_str': 'Providing service',
    'ED_DEPART_DATE_DATETIME_str': 'Patient departed',
    'ED_ADMIT_DECISION_DATE_DATETIME_str': 'Making admit decision',
    'ED_ADMIT_DATE_DATETIME_str': 'Admitting patient',  # Assuming you want to rename this as well, to keep consistency
    'ED_DISCHARGE_DATE_DATETIME_str': 'Patient discharge'
}

# Replace the values in the 'Activity' column using the mapping
df_melt_sorted['Activity'] = df_melt_sorted['Activity'].replace(value_mapping)


In [ ]:
df_melt_sorted.head()

In [ ]:
df_melt_sorted

In [ ]:
# Assuming df_melt_sorted has columns 'VISIT_ID', 'Activity', and 'Start_Time'

# First, group by 'VISIT_ID' and then check for duplicated 'Start_Time' within each group
# The `transform` function with `duplicated` marks all duplicates (both first and subsequent occurrences) as True within each group
duplicates_mask = df_melt_sorted.groupby('VISIT_ID')['Start_Time'].transform(lambda x: x.duplicated(keep=False))

# Filter the DataFrame to only include rows where the 'Start_Time' is duplicated within the same 'VISIT_ID'
df_duplicates = df_melt_sorted[duplicates_mask]

# df_duplicates now contains instances where 'Start_Time' equals the start time of a different activity during the same visit


In [ ]:
df_duplicates['FACILITY_NAME'].unique()

In [ ]:
# Save the filtered DataFrame to a CSV file

df_melt_sorted.to_csv('df_melt_sorted_fin.csv', index=False)

In [ ]:
df_melt_sorted.head()

In [ ]:
df_melt_sorted.info()

In [ ]:
filtered_df = df_visits[df_visits['VISIT_ID'] == "VISIT2212009"]


print(filtered_df)